# Tree Canopy Detection with YOLOv8

In [1]:
# If packages are missing, uncomment the line below
#!pip install ultralytics opencv-python matplotlib

In [2]:
from pathlib import Path
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
model = YOLO("yolov8n.pt")  # start from pretrained weights; swap to 'yolov8n.yaml' to train from scratch

In [4]:
data_yaml = Path("Tree-Top-View-6/data.yaml").resolve()
assert data_yaml.exists(), f"Data config not found at {data_yaml}"
train_settings = {
    "data": str(data_yaml),
    "epochs": 80,
    "imgsz": 640,
    "batch": 16,
    "patience": 20,
    "project": "runs/tree_canopy",
    "name": "yolov8n-tree",
    "device": 0,
}
train_results = model.train(**train_settings)

Ultralytics 8.3.223  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\TreeSense\Tree-Top-View-6\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n-tree2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.

In [5]:
metrics = model.val(split="val")
metrics.box.map, metrics.box.map50

Ultralytics 8.3.223  Python-3.11.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 987.8258.3 MB/s, size: 69.0 KB)
val: Scanning D:\TreeSense\Tree-Top-View-6\valid\labels.cache... 118 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 118/118  0.0s
val: Fast image access  (ping: 0.00.0 ms, read: 987.8258.3 MB/s, size: 69.0 KB)
val: Scanning D:\TreeSense\Tree-Top-View-6\valid\labels.cache... 118 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 118/118  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.8it/s 4.4s0.4ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.8it/s 4.4s
                   all        118       3189      0.624      0.61

(np.float64(0.2783837938885765), np.float64(0.6336062578120512))

In [6]:
test_dir = Path("Tree-Top-View-6/test/images")
output_dir = Path("runs/tree_canopy/predictions")
output_dir.mkdir(parents=True, exist_ok=True)
names = model.names if isinstance(model.names, dict) else dict(enumerate(model.names))
prediction_stream = model.predict(source=str(test_dir), conf=0.25, imgsz=640, save=False, stream=True)
for idx, result in enumerate(prediction_stream):
    image = cv2.imread(result.path)
    if image is None:
        continue
    boxes = result.boxes
    if boxes is None:
        continue
    xyxy = boxes.xyxy.cpu().numpy().astype(int)
    confidences = boxes.conf.cpu().numpy()
    classes = boxes.cls.cpu().numpy().astype(int)
    for (x1, y1, x2, y2), cls_id, conf in zip(xyxy, classes, confidences):
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{names.get(cls_id, 'tree')} {conf:.2f}"
        cv2.putText(image, label, (x1, max(y1 - 10, 0)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
    save_path = output_dir / f"{Path(result.path).stem}_pred.jpg"
    cv2.imwrite(str(save_path), image)
    if idx < 3:
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis("off")
plt.show()


image 1/60 d:\TreeSense\Tree-Top-View-6\test\images\13_jpg.rf.0d0ec444b9911cc17b09f7a62f3104e8.jpg: 640x640 27 tree-tops, 25.9ms
image 1/60 d:\TreeSense\Tree-Top-View-6\test\images\13_jpg.rf.0d0ec444b9911cc17b09f7a62f3104e8.jpg: 640x640 27 tree-tops, 25.9ms
image 2/60 d:\TreeSense\Tree-Top-View-6\test\images\17_jpg.rf.a155f68af1ff11a50d208c47c2a9a7eb.jpg: 640x640 54 tree-tops, 25.8ms
image 2/60 d:\TreeSense\Tree-Top-View-6\test\images\17_jpg.rf.a155f68af1ff11a50d208c47c2a9a7eb.jpg: 640x640 54 tree-tops, 25.8ms
image 3/60 d:\TreeSense\Tree-Top-View-6\test\images\2018_SJER_3_252000_4106000_image_234_jpeg.rf.5bf0ffaf21b4ca17d380f3a79550f4c8.jpg: 640x640 2 tree-tops, 26.1ms
image 3/60 d:\TreeSense\Tree-Top-View-6\test\images\2018_SJER_3_252000_4106000_image_234_jpeg.rf.5bf0ffaf21b4ca17d380f3a79550f4c8.jpg: 640x640 2 tree-tops, 26.1ms
image 4/60 d:\TreeSense\Tree-Top-View-6\test\images\2018_SJER_3_252000_4106000_image_326_jpeg.rf.973c90998bb890b51465ab12a6fe66e5.jpg: 640x640 (no detections

<Figure size 800x800 with 1 Axes>

<Figure size 800x800 with 1 Axes>

<Figure size 800x800 with 1 Axes>

In [7]:
def detect_trees_in_image(image_source, conf=0.25, imgsz=640, show=True, save_path=None):
    """Run the trained YOLO model on a single image and draw green boxes for each detected tree."""
    results = model.predict(source=image_source, conf=conf, imgsz=imgsz, save=False, verbose=False)
    if not results:
        raise RuntimeError("Model did not return any predictions; check the input source.")
    result = results[0]
    annotated = result.orig_img.copy()
    boxes = result.boxes
    tree_count = 0
    if boxes is not None and len(boxes) > 0:
        tree_count = len(boxes)
        xyxy = boxes.xyxy.cpu().numpy().astype(int)
        confidences = boxes.conf.cpu().numpy()
        classes = boxes.cls.cpu().numpy().astype(int)
        names = model.names if isinstance(model.names, dict) else dict(enumerate(model.names))
        for (x1, y1, x2, y2), cls_id, conf_score in zip(xyxy, classes, confidences):
            cv2.rectangle(annotated, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f"{names.get(cls_id, 'tree')} {conf_score:.2f}"
            cv2.putText(annotated, label, (x1, max(y1 - 10, 0)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(save_path), annotated)
    if show:
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        plt.title(f"Trees detected: {tree_count}")
        plt.axis("off")
        plt.show()
    return annotated, tree_count

In [8]:
detect_trees_in_image('./hq720.jpg', save_path='./hq720_detected.jpg')

<Figure size 1000x1000 with 1 Axes>

(array([[[ 70,  91,  83],
         [ 70,  91,  83],
         [ 70,  91,  82],
         ...,
         [124, 165, 188],
         [124, 165, 188],
         [124, 165, 188]],
 
        [[ 70,  91,  83],
         [ 70,  91,  83],
         [ 71,  92,  83],
         ...,
         [125, 166, 189],
         [125, 166, 189],
         [125, 166, 189]],
 
        [[ 70,  91,  83],
         [ 71,  92,  84],
         [ 71,  92,  83],
         ...,
         [126, 167, 190],
         [126, 167, 190],
         [126, 167, 190]],
 
        ...,
 
        [[ 87, 115, 126],
         [ 79, 110, 119],
         [ 74, 105, 114],
         ...,
         [ 44,  70,  77],
         [ 45,  73,  80],
         [ 70,  98, 105]],
 
        [[ 77, 108, 117],
         [ 76, 108, 114],
         [ 73, 107, 113],
         ...,
         [ 58,  84,  91],
         [ 58,  86,  93],
         [ 56,  85,  92]],
 
        [[ 77, 109, 115],
         [ 74, 108, 114],
         [ 73, 108, 112],
         ...,
         [ 68,  94, 101],
  